# Fast Prompting – Con base de conocimiento

Este notebook muestra la misma consulta pero usando un prompt optimizado:
- Rol definido
- Contexto desde la base de conocimiento (CSV)
- Reglas de salida (≤150 palabras, tono institucional, fuente, derivación)
- Posibilidad de complementar con referencia libre (ej. definiciones generales)

In [10]:
import pandas as pd
import re
from textwrap import dedent

import csv
from pathlib import Path

ruta = Path("../data/base_conocimiento_afiliaciones_clean.csv")  # o tu CSV
with open(ruta, encoding="utf-8") as f:
    r = csv.reader(f)
    esperado = None
    for i, fila in enumerate(r, start=1):
        if i == 1:
            esperado = len(fila)
        if len(fila) != esperado:
            print(f"Línea {i} con {len(fila)} columnas (esperado {esperado}):", fila)

# Cargar base de conocimiento
df = pd.read_csv("../data/base_conocimiento_afiliaciones_clean.csv", dtype=str, keep_default_na=False)

# Filtrar por estado
df = df[df["estado"].str.lower().isin(["vigente","en revisión"])].copy()

# Normalizar columnas
for col in ["id","titulo","contenido","respuesta_validada","palabras_clave"]:
    if col in df.columns:
        df[col] = df[col].fillna("").astype(str)

# Funciones auxiliares
def dividir_en_tokens(texto: str):
    texto = texto.lower()
    return [t for t in re.split(r"[^a-záéíóúñü0-9]+", texto) if t]

def calcular_relevancia(tokens_consulta, fila):
    puntaje = 0
    puntaje += 3 * len(set(tokens_consulta) & set(dividir_en_tokens(fila.get("palabras_clave",""))))
    puntaje += 2 * len(set(tokens_consulta) & set(dividir_en_tokens(fila.get("titulo",""))))
    puntaje += 1 * len(set(tokens_consulta) & set(dividir_en_tokens(fila.get("contenido",""))))
    return puntaje

def buscar_faq(consulta: str, tabla: pd.DataFrame):
    toks_consulta = dividir_en_tokens(consulta)
    puntuadas = [(calcular_relevancia(toks_consulta, fila), idx) for idx, fila in tabla.iterrows()]
    puntuadas = [(p,i) for p,i in puntuadas if p>0]
    if not puntuadas:
        return None
    puntuadas.sort(reverse=True)
    return tabla.loc[puntuadas[0][1]]



def construir_prompt(fila, pregunta: str):
    base = (fila.get("respuesta_validada") or fila.get("contenido") or fila.get("titulo","")).strip()

    return dedent(f"""
    Rol: asistente interno de IOMA. Tono institucional y uso interno por agentes.

      Reglas:
    - Usá SIEMPRE la BASE como núcleo. No inventes ni contradigas la BASE.
    - Al final, incluí SIEMPRE UNO O DOS  parrafos de “Contexto: …”.
    - El contexto debe aportar valor (definición/propósito/criterio general vinculado a la BASE), evitando obviedades o frases genéricas.
      Si realmente no hay nada útil, escribí: “Contexto: no aplica”.
    - Si la BASE es breve, podés ampliarla con 1–2 aclaraciones generales (p. ej., propósito del documento o condición típica),
      sin agregar plazos/condiciones que no figuren en la BASE.
    - No cites URLs ni números de norma que no estén en la BASE. Si falta un dato, escribí “No consta en la normativa adjunta”
      y derivá a Afiliaciones (SLA: 24 h).
    - Extensión total ≤350 palabras.

    Formato de salida:
    - Lista/pasos/checklist (según corresponda).
    - Cierre: "Fuente: base de conocimiento vigente".
    - Línea final: "Contexto: …"

    Pregunta del agente:
    "{pregunta}"

    CONTEXTO – BASE (obligatorio):
    \"\"\"{base}\"\"\"
    """).strip()




### Ejemplo de consulta

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

consulta = "¿Qué documentación necesito para afiliar a un conviviente con union convivencial?"

fila_sel = buscar_faq(consulta, df)
if fila_sel is None:
    print("No se encontró una FAQ relevante. Derivar a Afiliaciones (SLA 24 h).")
else:
    print("FAQ seleccionada:", fila_sel.get("id","(sin id)"), "-", fila_sel.get("titulo","(sin título)"))

    prompt = construir_prompt(fila_sel, consulta)

    # Igual que en 04: system = reglas+BASE (tu prompt), user = consulta
    resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": construir_prompt(fila_sel, consulta)},
        {"role": "user", "content": consulta},
    ],
    temperature=0.3,
    max_tokens=400,
)
print("\n--- RESPUESTA DEL MODELO ---\n")
print(resp.choices[0].message.content)


    
  

FAQ seleccionada: faq_001a - Afiliación de conviviente con Unión Convivencial

--- RESPUESTA DEL MODELO ---

Para afiliar a un conviviente registrado en el Registro de las Personas mediante Unión Convivencial, se debe presentar la siguiente documentación:

1. Ficha afiliatoria completa con teléfono y mail.
2. Credencial y certificación afiliatoria.
3. Último recibo de sueldo (para afiliados obligatorios).
4. DNI del afiliado/a y de su pareja con idéntico domicilio.
5. Acta o partida de unión convivencial.
6. Certificación negativa de ANSES de los últimos dos períodos.
7. En caso de registrar Obra Social Obligatoria, corresponde adjuntar planilla de aceptación de descuento.

Es importante destacar que el trámite solo se admite si el titular no tiene cónyuge/conviviente ya afiliado a su cargo.

Fuente: base de conocimiento vigente.  
Contexto: La afiliación de convivientes mediante Unión Convivencial busca reconocer y garantizar el acceso a los beneficios de salud y asistencia social a p